In [12]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from modules.BatchPitNormalization import BatchPitNorm1d
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

Using cuda device


In [13]:
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


infrared_thermography_temperature = fetch_ucirepo(id=925) 


X_df = infrared_thermography_temperature.data.features 
y_df = infrared_thermography_temperature.data.targets 

le_gender = LabelEncoder()
le_age = LabelEncoder()
le_ethnicity = LabelEncoder()

X_df['Gender_Encoded'] = le_gender.fit_transform(X_df['Gender'])
X_df['Age_Encoded'] = le_age.fit_transform(X_df['Age'])
X_df['Ethnicity_Encoded'] = le_ethnicity.fit_transform(X_df['Ethnicity'])

X_df = X_df.drop('Gender', axis=1)
X_df = X_df.drop('Age', axis=1)
X_df = X_df.drop('Ethnicity', axis=1)


X = X_df.values
Y = y_df.values[:, 1].reshape(-1, 1)


Y=Y[~np.isnan(X).any(axis=1)]
X=X[~np.isnan(X).any(axis=1)]

print(np.mean(Y))

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, train_size = 0.7, shuffle = True)

scalerX = StandardScaler()
scalerY = StandardScaler()
# scale
scaler_x = scalerX.fit(X_train_raw)
scaler_y = scalerY.fit(y_train_raw)

X_train = scaler_x.transform(X_train_raw)
X_test = scaler_x.transform(X_test_raw)


y_train = scaler_y.transform(y_train_raw)
y_test = scaler_y.transform(y_test_raw)

X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device).reshape(-1,1)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device).reshape(-1,1)


37.028163064833


/tmp/ipykernel_408606/1941237738.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['Gender_Encoded'] = le_gender.fit_transform(X_df['Gender'])
/tmp/ipykernel_408606/1941237738.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['Age_Encoded'] = le_age.fit_transform(X_df['Age'])


In [14]:
#without bn
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
 
    def forward(self, x):
        x = self.layers(x)
        return x

In [15]:
#with bn on first
class NeuralNetwork_bn(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork_bn, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
 
    def forward(self, x):
        x = self.layers(x)
        return x
    
class NeuralNetwork_bn2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork_bn2, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
 
    def forward(self, x):
        x = self.layers(x)
        return x

In [16]:
# with batch pit
class NeuralNetwork_pit(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes,num_pit_samples,take_num_samples_when_full,normal_backtransform,trainable_bandwidth):
        super(NeuralNetwork_pit, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x
    
class NeuralNetwork_pit2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes,num_pit_samples,take_num_samples_when_full,normal_backtransform,trainable_bandwidth):
        super(NeuralNetwork_pit2, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x
    
class NeuralNetwork_pit3(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes,num_pit_samples,take_num_samples_when_full,normal_backtransform,trainable_bandwidth):
        super(NeuralNetwork_pit3, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            BatchPitNorm1d(num_features=hidden_size,num_pit_samples=num_pit_samples,dev=device,take_num_samples_when_full=take_num_samples_when_full,normal_backtransform=normal_backtransform,trainable_bandwidths=trainable_bandwidth),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [17]:
def get_train_test_data(X,Y,grid_search: bool=False):
    if grid_search:
        X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, train_size = 0.7, random_state=1)
    else:
        X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, train_size = 0.7, shuffle = True)
    #scalers
    scaler_x = StandardScaler().fit(X_train_raw)
    scaler_y = StandardScaler().fit(y_train_raw)

    #scale data
    X_train = scaler_x.transform(X_train_raw)
    X_test = scaler_x.transform(X_test_raw)
    y_train = scaler_y.transform(y_train_raw)
    y_test = scaler_y.transform(y_test_raw)
    #make tensors
    X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
    y_train = torch.tensor(y_train, dtype=torch.float32, device=device).reshape(-1,1)
    X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_test = torch.tensor(y_test, dtype=torch.float32, device=device).reshape(-1,1)

    return X_train, y_train, X_test, y_test, scaler_x, scaler_y

In [18]:
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error
def train_model(model,grid_search: bool=False):
    X_train,y_train,X_test,y_test,scaler_x,scaler_y = get_train_test_data(X,Y,grid_search)
    
    input_size = 8
    hidden_size = 32
    num_classes = 1
    learning_rate = 0.001 
    n_epochs = 30 
    batch_size = 64
    model.to(device)
    model.cuda()
    lossFunction = nn.MSELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=batch_size)
    model.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        for Xbatch, ybatch in loader:
            
            y_pred = model(Xbatch)
            loss = lossFunction(y_pred, ybatch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            running_loss += loss.item()

    model.eval()
    y_pred_eval = model(X_test)
    y_pred_eval = y_pred_eval.detach().cpu().numpy()
    y_pred_scaled = scaler_y.inverse_transform(y_pred_eval)
    y_test_eval = y_test.detach().cpu().numpy()
    y_test_scaled = scaler_y.inverse_transform(y_test_eval)
    r2 = r2_score(y_test_scaled,y_pred_scaled)
    mae = mean_absolute_error(y_test_scaled,y_pred_scaled)
    rmse = root_mean_squared_error(y_test_scaled,y_pred_scaled)
    #print("R2-score: ",r2)
    #print("MAE: ", mae)
    #print("RMSE: ", rmse)
    return r2, mae,rmse

In [22]:
import pandas as pd
def grid_search():
    from itertools import product
    # Hyperparameter grid
    param_grid = {
        'num_pit_samples': [50, 100, 150, 250, 500, 1000],
        'take_num_samples_when_full': [0, 1, 2, 3, 5, 10],
        'normal_backtransform': [True, False],
        'trainable_bandwidths': [True, False]
    }
    results = np.array(("mae","r2","num_pit_samples","take_num_samples_when_full","normal_backtransform","trainable_bandwidth","model_id"))
    model_id = "model 3"
    hidden_size = 32
    input_size = 33
    num_classes = 1
    best_params = []

    for params in product(*param_grid.values()):
        num_pit_samples, take_num_samples_when_full, normal_backtransform, trainable_bandwidths = params
        model = NeuralNetwork_pit3(input_size=input_size,
                                hidden_size=hidden_size,
                                num_classes=num_classes,
                                num_pit_samples=num_pit_samples,
                                normal_backtransform=normal_backtransform,
                                take_num_samples_when_full=take_num_samples_when_full,
                                trainable_bandwidth=trainable_bandwidths
                                )
        # train model
        try:
            r2, mae,rmse = train_model(model,grid_search=True)
            
            if best_params == []:
                best_params.append(mae)
                best_params.append(params)
            elif mae < best_params[0]:
                best_params[0] = mae
                best_params[1] = params

            grid_res = np.array((mae,r2, params[0],params[1],params[2],params[3],model_id))
            results = np.vstack((results,grid_res))
            print(best_params, params, mae)
            #print(results)
        except:
            return results
    print("best!!! ", best_params)
    return results

results_grid = grid_search()

# convert array into dataframe 
DF = pd.DataFrame(results_grid) 
  
# save the dataframe as a csv file 
DF.to_csv("temp_model3.csv")

[0.18723373, (50, 0, True, True)] (50, 0, True, True) 0.18723373
[0.18723373, (50, 0, True, True)] (50, 0, True, False) 0.20371608
[0.18723373, (50, 0, True, True)] (50, 0, False, True) 0.20755868
[0.18723373, (50, 0, True, True)] (50, 0, False, False) 0.22728047
[0.18723373, (50, 0, True, True)] (50, 1, True, True) 0.19017674
[0.18723373, (50, 0, True, True)] (50, 1, True, False) 0.22089775
[0.18723373, (50, 0, True, True)] (50, 1, False, True) 0.22905916
[0.18723373, (50, 0, True, True)] (50, 1, False, False) 0.23737912
[0.18723373, (50, 0, True, True)] (50, 2, True, True) 0.2116947
[0.18723373, (50, 0, True, True)] (50, 2, True, False) 0.1961536
[0.18723373, (50, 0, True, True)] (50, 2, False, True) 0.23323366
[0.18723373, (50, 0, True, True)] (50, 2, False, False) 0.24001305
[0.18723373, (50, 0, True, True)] (50, 3, True, True) 0.20962101
[0.18723373, (50, 0, True, True)] (50, 3, True, False) 0.2148238
[0.18723373, (50, 0, True, True)] (50, 3, False, True) 0.23517145
[0.18723373, (